In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

In [9]:
df = pd.read_csv('passing-grade.csv')

df.head(5)

,NO,PTN,KODE PRODI,NAMA PRODI,RATAAN,S.BAKU,MIN,MAX
0,1,UNIVERSITAS INDONESIA,3211015,PENDIDIKAN DOKTER,747.93,19.63,724.38,798.55
1,2,UNIVERSITAS AIRLANGGA,3811014,PENDIDIKAN DOKTER,726.94,21.29,702.54,808.01
2,3,UNIVERSITAS PADJADJARAN,3331013,PENDIDIKAN DOKTER,717.94,18.69,694.17,780.52
3,4,UNIVERSITAS GADJAH MADA,3611066,KEDOKTERAN,725.21,27.08,689.05,809.84
4,5,UNIVERSITAS INDONESIA,3211166,ILMU KOMPUTER,712.41,24.13,685.96,798.66


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   NO          500 non-null    int64  
 1   PTN         500 non-null    object 
 2   KODE PRODI  500 non-null    int64  
 3   NAMA PRODI  500 non-null    object 
 4   RATAAN      500 non-null    float64
 5   S.BAKU      500 non-null    float64
 6   MIN         500 non-null    float64
 7   MAX         500 non-null    float64
dtypes: float64(4), int64(2), object(2)
memory usage: 31.4+ KB


In [11]:
print(len(df['PTN'].unique()))

50


In [12]:
X = df[['RATAAN', 'S.BAKU', 'MIN', 'MAX']]
y = df['PTN']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_1 = model.predict(X_train)

print("Confusion Matrix:")
print(confusion_matrix(y_train, y_1))

print("\nClassification Report:")
print(classification_report(y_train, y_1))

accuracy = accuracy_score(y_train, y_1)
f1 = f1_score(y_train, y_1, average='weighted')
print(f"\nAccuracy Score: {accuracy*100:.0f}%")
print(f"F1 Score: {f1*100:.0f}")

Confusion Matrix:
[[27  0  0 ...  0  0  0]
 [ 0 10  0 ...  0  0  0]
 [ 0  0 22 ...  0  0  0]
 ...
 [ 0  0  0 ...  5  0  0]
 [ 0  0  0 ...  0  6  0]
 [ 0  0  0 ...  0  0  9]]

Classification Report:
                                         precision    recall  f1-score   support

               INSTITUT PERTANIAN BOGOR       1.00      1.00      1.00        27
             INSTITUT TEKNOLOGI BANDUNG       1.00      1.00      1.00        10
    INSTITUT TEKNOLOGI SEPULUH NOPEMBER       1.00      1.00      1.00        22
                  UNIVERSITAS AIRLANGGA       1.00      1.00      1.00        15
                    UNIVERSITAS ANDALAS       1.00      1.00      1.00        10
                   UNIVERSITAS BENGKULU       1.00      1.00      1.00         1
                  UNIVERSITAS BRAWIJAYA       1.00      1.00      1.00        29
               UNIVERSITAS CENDERAWASIH       1.00      1.00      1.00         1
                 UNIVERSITAS DIPONEGORO       1.00      1.00      1.00  

In [15]:
y_pred = model.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"\nAccuracy Score: {accuracy*100:.0f}%")
print(f"F1 Score: {f1*100:.0f}")

Confusion Matrix:
[[3 0 1 ... 1 0 1]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]]

Classification Report:
                                             precision    recall  f1-score   support

                   INSTITUT PERTANIAN BOGOR       0.21      0.33      0.26         9
                 INSTITUT TEKNOLOGI BANDUNG       0.14      0.17      0.15         6
        INSTITUT TEKNOLOGI SEPULUH NOPEMBER       0.10      0.11      0.11         9
                      UNIVERSITAS AIRLANGGA       0.00      0.00      0.00         1
                        UNIVERSITAS ANDALAS       1.00      0.20      0.33         5
                      UNIVERSITAS BRAWIJAYA       0.20      0.25      0.22         4
                     UNIVERSITAS DIPONEGORO       0.00      0.00      0.00         7
                    UNIVERSITAS GADJAH MADA       0.35      0.40      0.38        15
                      UNIVERSITAS INDONESIA       0.57      0.44      0.

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric